In [17]:
# Compares values in two files and uses fuzzy matching to help a human identify matching records.

In [67]:
from fuzzywuzzy import fuzz, process
import pandas as pd

In [68]:
df_1 = pd.read_csv('../mqdq/data/dll-works-concat-fuzzy.csv')
df_2 = pd.read_csv('../mqdq/data/mqdq-works-fuzzy.csv')

In [69]:
df_1['match']

0                                a. furius antias|carmina
1                              a. persius flaccus|saturae
2       achilleus, episcopus spoletanus|epigramma in e...
3                      aemilius macer, veronensis|carmina
4       aemilius macer, veronensis|fragmentum a morel ...
5                                albinouanus pedo|carmina
6                                       albinus|de metris
7                         albinus|rerum romanarum liber i
8                albius tibullus|carmina tibulliana [sp.]
9                                 albius tibullus|elegiae
10      alcimus ecdicius auitus, episcopus viennensis|...
11      alcimus ecdicius auitus, episcopus viennensis|...
12      alcimus ecdicius auitus, episcopus viennensis|...
13      alcimus ecdicius auitus, episcopus viennensis|...
14      alcimus ecdicius auitus, episcopus viennensis|...
15      alcimus ecdicius auitus, episcopus viennensis|...
16      alcimus ecdicius auitus, episcopus viennensis|...
17      alcimu

In [70]:
df_2.columns

Index(['DLLAuthor', 'DLL Work', 'mqdqauthor', 'mqdqtitle', 'Unnamed: 4',
       'edition', 'URL'],
      dtype='object')

In [71]:
df_1['match'] = df_1.apply(lambda x: "|".join((str(x['mqdqdauthor']).strip().lower(), str(x['DLLWork']).strip().lower())), axis=1)

In [72]:
df_2['match'] = df_2.apply(lambda x: "|".join((str(x['mqdqauthor']).strip().lower(), str(x['mqdqtitle']).strip().lower())), axis=1)

In [73]:
final_results = []
for item_1 in list(df_2['match']):
    max_score = 0
    result = ""
    for item_2 in list(df_1['match']):
        score = fuzz.ratio(item_1, item_2)
        if score > max_score:
            max_score = score
            result = item_1, item_2, score
    final_results.append(result)

In [74]:
df_results = pd.DataFrame(final_results, columns=['match', 'dll', 'score'])

In [75]:
len(df_results)

584

In [76]:
df_results

,match,dll,score
0,"alcuinus magister, abbas turonensis|carmina","alcuinus magister, abbas turonensis|carmina",100
1,"alcuinus magister, abbas turonensis|carmina rh...","alcuinus magister, abbas turonensis|carmina rh...",99
2,anicius manlius seuerinus boethius|consolatio ...,anicius manlius seuerinus boethius|categoriae,82
3,anicius manlius seuerinus boethius|elogia,anicius manlius seuerinus boethius|topica,93
4,"lucius apuleius, madaurensis|carminum fragmenta","lucius apuleius, madaurensis|fragmenta",89
5,"lucius apuleius, madaurensis|*fragmentum poeti...","lucius apuleius, madaurensis|fragmenta",69
6,"arator, subdiaconus ecclesiae romanae|epistula...","arator, subdiaconus ecclesiae romanae|epistola...",91
7,"arator, subdiaconus ecclesiae romanae|epistula...","arator, subdiaconus ecclesiae romanae|epistola...",92
8,"arator, subdiaconus ecclesiae romanae|de actib...","arator, subdiaconus ecclesiae romanae|historia...",82
9,"arator, subdiaconus ecclesiae romanae|epistula...","arator, subdiaconus ecclesiae romanae|epistola...",98


In [77]:
df_merged = pd.merge(df_results, df_2, on='match')

In [78]:
df_merged.sort_values(by=['score'], ascending=False).to_csv("../mqdq/data/works_fuzzy_match_scores.csv")